In [ ]:
from common import utils
import random
u=utils(False)

# Les flux

Les programmes informatiques (applications) communiquent avec des moyens divers, et des interlocuteurs divers.

Ils communiquent avec :

  * des utilisateurs locaux (applications)
  * des utilisateurs distants (client et serveur)
  * des bases de données
  * des écrans (et autres dispositifs d'entrée-sortie)
  * d'autres ordinateurs

La réalité est que les seuls programmes qui ont un effet sont ceux qui permettent des entrées-sorties.
Tous les autres programmes ne font que communiquer avec d'autres programmes.

On distingue plusieurs types d'entrée ou sortie du système :

  * Stockage de masse
  * Réseau
  * Dispositifs physiques d'entrée-sortie (du clavier au bras mécanique en passant par les écrans et les moteurs de fusée).

Le shell permet de contrôler certaines communications entre les processus et autres intervenants.

In [ ]:
from graphviz import Graph

dot = Graph(comment='Tree')
processx = [ "écran","réseau", "clavier", "souris", "carte son", "disque dur"]
process = []
for i in range(0,8):
    process.append(str(random.randint(0,20000)+1024))
for i in process+processx:
    dot.node(i)

xorg = None

for i in process.copy():
    if i in process:
        process.remove(i)
    l = processx + process
    connected = False
    if xorg == None:
        dot.edge(i,"clavier")
        dot.edge(i,"souris")
        dot.edge(i,"écran")
        processx.remove("clavier")
        processx.remove("souris")
        processx.remove("écran")
        l = processx + process
        processx.append(i)
        xorg = i
        connected = True
    else:
        if random.random()<.8:
            m = xorg
            l.remove(m)
            dot.edge(i,m)
            connected = True
        if random.random()<.9:
            m = "disque dur"
            l.remove(m)
            dot.edge(i,m)
            connected = True
    while len(l)>0 and (random.random()<.3 or not connected):
        m = random.choice(l)
        l.remove(m)
        dot.edge(i,m)
display(dot)

Les communications qui nous intéressent sont les communications qui ont lieu par un certain type d'interface, les interfaces de type *fichier*. S'il paraît évident que les interfaces de type fichier sont utilisées par le disque dur, elles le sont aussi pour un grand nombre d'autres périphériques d'entrée-sortie dans la famille des systèmes d'exploitation Unix/Linux.

## Un processus, des flux

Chaque processus fait appel au système d'exploitation pour communiquer avec le reste du système. Dans le cadre de la communication par un flux de type *fichier*, cette communication est codée par un certain nombre d'informations qui sont toutes regroupées dans un **descripteur de fichier** (en anglais, *file descriptor* ou *fd*). Chaque processus peut avoir au plus un certain nombre de descripteurs de fichiers ouverts en même temps qui est fixé par le système.

Chaque processus peut ainsi être vu comme une petite cellule, qui pour communiquer dispose de fenêtres qu'elle peut connecter à d'autres dispositifs (en général, un *fichier* sur un disque dur, mais nous verrons plus loin que ce n'est pas du tout le seul cas). Ces fenêtres, à leur ouverture, sont mises en mode *lecture*, *écriture*, ou encore bidirectionnel (*lecture et écriture*).

In [ ]:
%%sh
#! /bin/sh
# L'explication de cette syntaxe arrive plus loin !
# ulimit -n permet de lister le nombre de descripteurs de fichiers simultanés autorisés
echo "Il est possible d'ouvrir au plus simultanément $(ulimit -n) fichiers sur ce système."

In [ ]:
u.xtoggle()